# MNIST CNN & required packages

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = input_data.read_data_sets('./dataset/MNIST_data/', one_hot=True)

Extracting ./dataset/MNIST_data/train-images-idx3-ubyte.gz
Extracting ./dataset/MNIST_data/train-labels-idx1-ubyte.gz
Extracting ./dataset/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./dataset/MNIST_data/t10k-labels-idx1-ubyte.gz


# Block strategy

### Initial weights/biases

In [3]:
def init_weights(shape):
    init_random_dist = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(init_random_dist)

In [4]:
def init_biases(shape):
    init_random_dist = tf.random_normal(shape, stddev=0.1)
    return tf.Variable(init_random_dist)

### Fully connected layer

In [5]:
def full_connected_layer(input_layer, size, actfn=tf.nn.relu):
    input_size = int(input_layer.get_shape()[1])
    W = init_weights([input_size, size])
    b = init_biases([size])
    z = tf.matmul(input_layer, W) + b
    return actfn(z) if actfn else z

### Dropout layer

In [6]:
def dropout_layer(input_layer, keep_prob):
    return tf.nn.dropout(input_layer, keep_prob=keep_prob)

### Convolutional layer

In [7]:
def conv2d(x, W, strides, padding):
    # x: [batch, h, w, channels]
    # W: [filter h, filter w, channels in, channels out]
    return tf.nn.conv2d(x, W, strides=strides, padding=padding)

In [8]:
def max_pool(x, ksize, strides, padding):
    # x: [batch, h, w, c]
    return tf.nn.max_pool(x, ksize=ksize, strides=strides, padding=padding)

In [9]:
def max_pool_2by2(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME'):
    return max_pool(x, ksize, strides, padding)

In [10]:
def convolutional_layer(input_layer, shape, actfn=tf.nn.relu, strides=[1, 1, 1, 1], padding='SAME'):
    W = init_weights(shape)
    b = init_biases([shape[3]])
    z = conv2d(input_layer, W, strides, padding) + b
    return actfn(z) if actfn else z

# Architecture

### Placeholder

In [11]:
X = tf.placeholder(tf.float32, shape=[None, 784])

In [12]:
y_true = tf.placeholder(tf.float32, shape=[None, 10])

In [13]:
keep_prob = tf.placeholder(tf.float32)

In [14]:
Xi = tf.reshape(X, [-1, 28, 28, 1])  

# Layers

In [15]:
conv_1 = convolutional_layer(Xi, shape=[5, 5, 1, 32])
conv_1_pooling = max_pool_2by2(conv_1)
print(conv_1_pooling)

Tensor("MaxPool:0", shape=(?, 14, 14, 32), dtype=float32)


In [16]:
conv_2 = convolutional_layer(conv_1_pooling, shape=[5, 5, 32, 64])
conv_2_pooling = max_pool_2by2(conv_2)
print(conv_2_pooling)

Tensor("MaxPool_1:0", shape=(?, 7, 7, 64), dtype=float32)


In [17]:
conv_2_flat = tf.reshape(conv_2_pooling, [-1, 7*7*64])
print(conv_2_flat)

Tensor("Reshape_1:0", shape=(?, 3136), dtype=float32)


In [18]:
full_layer_1 = full_connected_layer(conv_2_flat, 1024)

In [19]:
full_layer_1_dropout = dropout_layer(full_layer_1, keep_prob)

In [20]:
y_pred = full_connected_layer(full_layer_1_dropout, 10, None)

# Loss function

In [21]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred))

# Optimizer

In [22]:
optimizer = tf.train.AdamOptimizer(learning_rate=1e-3)
train = optimizer.minimize(loss)

# Prepare for learning

In [23]:
init = tf.global_variables_initializer()

In [24]:
steps = 300
batch_sz = 50
dropout = 0.5

with tf.Session() as sess:
    sess.run(init)
    
    for step in range(steps):
        
        x_train_batch, y_train_batch = mnist.train.next_batch(batch_sz)

        sess.run(train, feed_dict={X: x_train_batch, y_true: y_train_batch, keep_prob:dropout})

        if step % 100 == 0:
            matches = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y_true, 1))
            accuracy = tf.reduce_mean(tf.cast(matches, tf.float32))
            
            x_test = mnist.test.images
            y_test = mnist.test.labels
            acc, lss, preds = sess.run([accuracy, loss, y_pred], feed_dict={X: x_test, y_true: y_test, keep_prob:dropout})
            print('ON STEM: {:05d}    ACCURACY: {:.5f}    LOSS: {:.5f}'.format(step+1, acc, lss))

ON STEM: 00001    ACCURACY: 0.14240    LOSS: 7.74103
ON STEM: 00101    ACCURACY: 0.93100    LOSS: 0.22876
ON STEM: 00201    ACCURACY: 0.95470    LOSS: 0.14741


In [25]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import classification_report

%matplotlib inline

In [26]:
predictions = np.argmax(preds, axis=1)
y_test = np.argmax(y_test, axis=1)

In [27]:
print(classification_report(y_test, predictions))

             precision    recall  f1-score   support

          0       0.99      0.95      0.97       980
          1       0.99      0.97      0.98      1135
          2       0.93      0.97      0.95      1032
          3       0.97      0.93      0.95      1010
          4       0.97      0.97      0.97       982
          5       0.97      0.94      0.96       892
          6       0.94      0.98      0.96       958
          7       0.97      0.93      0.95      1028
          8       0.93      0.93      0.93       974
          9       0.90      0.96      0.93      1009

avg / total       0.96      0.95      0.95     10000

